# Lab One: Exploring Table Data

#### By: David Hoffman, Jonas Moros, and Sully Billingsly

## 1. Business Understanding

Bitcoin is the longest running and most well-known cryptocurrency on the market.  This dataset is comprised of minute to minute updates of OHLC (Open, High, Low, Close), Volume in BTC and indicated currency, and weighted bitcoin price from the period of January 2012 to March of 2021.  The first collumn of each row contains a Unix timestamp which represents when the remainder of the datapoints in that row were collected.

This data was collected in order to understand various trends within the Bitcoin market and is used by many traders to train the machine learning algorithms that they use to automate the entirety of their daily crypocurrency trading.  Similarly to these cryptocurrency day traders, our prediction task is to predict whether or not the price of bitcoin will rise or fall on any given day.  

If we are able to generate a meaningful conclusion from this data the results we obtain could prove incredibly useful for nearly any third party from large corporations to small private investors.  Given the ease with which Bitcoin can be traded on public markets such as Coinbase or Robinhood, anyone who has access to these applications could use our conclusions to guide their bitcoin trades in a profitable direction.  

In order for our research to be useful for our needs as well as the needs of the general consumer, our conclusions would need to have at least an 80% accuracy rating.  We believe it is impossible to be 100% accurate with these predictions given the incredible volatility of the market we are trying to predict; however, if we are able to reach an 80% accuracy rating our predictions would start to give us a competitive edge within the market.  With 80% accuracy we would expext to accurately predict most of the big swings which are where we would do the majority of our more high-risk buys and sells.  Contrarily, we would expect the majority of our 20% error to come from days that were not predicted to move much either way and went slighly up when we expected it to go down or vice versa.  On these days we would plan on holding our assets anyway and therefore would hardly feel the negative impact of a prediction error.

Dataset URL: https://www.kaggle.com/mczielinski/bitcoin-historical-data

## 2. Data Understanding

### 2.1 Data Description

In [2]:
import pandas as pd
import numpy as np

# load bitcoin historical dataset into pandas and give it a color attribute
df = pd.read_csv('./bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')

# Tracks the overall performance for each day, will be converted into categorical data
df['Day_Stats'] = df['Open'] - df['Close']

#print number of rows in df to confirm all data is loaded in
print('Length of bitcoin dataset:', df.shape[0])

df.head()

Length of bitcoin dataset: 4857377


,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Day_Stats
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,0.0
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As seen in the code above, we created an additional ninth column which I have declared 'Day_Stats.'  This is a very important column as it will track the overall performance of each day by subtracting the opening price from the closing price resulting in the overall movement in price over the course of a single day.  We will treat this as categorical data and will convert the numeric answer into a string which will define the day as either being 'Big Up', 'Small Up', 'No Change', 'Small Down', or 'Big Down'. In addition to the 'Day_Stats' categorical feature, 'Open', 'Close', 'High', and 'Low' will be very important features for our analysis as well.  We do not expect to find as useful of a correlation within the 'Volume_(BTC)', 'Volume_(Currency)', and 'Weighted_Price' features although we still will try to gain as much insight as possible from all of the different features included in our dataframe.

In [3]:
# Numerical Data in the 'Day_Stats' column is converted into a string that enhances readibility
df.loc[df['Day_Stats'] > 3, 'DS_categorical'] = 'Big Up' 
df.loc[(df['Day_Stats'] <= 3) & (df['Day_Stats'] > 1), 'DS_categorical'] = 'Small Up' 
df.loc[(df['Day_Stats'] <= 1) & (df['Day_Stats'] >= -1), 'DS_categorical'] = 'No Change' 
df.loc[(df['Day_Stats'] < -1) & (df['Day_Stats'] >= -3), 'DS_categorical'] = 'Small Down' 
df.loc[df['Day_Stats'] < -3, 'DS_categorical'] = 'Big Down'

df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Day_Stats,DS_categorical
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,0.0,No Change
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The conversion of the 'Day_Stats' data from an integer to a string greatly enhances readability and makes the data easier to understand from an outside perspective.  This feature will likely be the most important out of all of our columns and will be a large indicator for which features create rises and falls in the value of Bitcoin. The converted feature can be seen next to the integer on the far right of the table.

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4857377 entries, 0 to 4857376
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Timestamp          int64  
 1   Open               float64
 2   High               float64
 3   Low                float64
 4   Close              float64
 5   Volume_(BTC)       float64
 6   Volume_(Currency)  float64
 7   Weighted_Price     float64
 8   Day_Stats          float64
 9   DS_categorical     object 
dtypes: float64(8), int64(1), object(1)
memory usage: 370.6+ MB
None


The .info() function is generally utilized to print a concise summary of a particular dataframe.  As you can see, all of my features are defined as floats except for the timestamp which is an integer and the day_stats which is an object string.

In [5]:
df.describe()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Day_Stats
count,4.857377e+06,3.613769e+06,3.613769e+06,3.613769e+06,3.613769e+06,3.613769e+06,3.613769e+06,3.613769e+06,3.613769e+06
mean,1.471301e+09,6.009024e+03,6.013357e+03,6.004488e+03,6.009014e+03,9.323249e+00,4.176284e+04,6.008935e+03,1.013512e-02
std,8.428019e+07,8.996247e+03,9.003521e+03,8.988778e+03,8.996360e+03,3.054989e+01,1.518248e+05,8.995992e+03,1.537593e+01
min,1.325318e+09,3.800000e+00,3.800000e+00,1.500000e+00,1.500000e+00,0.000000e+00,0.000000e+00,3.800000e+00,-1.186760e+03
25%,1.398179e+09,4.438600e+02,4.440000e+02,4.435200e+02,4.438600e+02,4.097759e-01,4.521422e+02,4.438306e+02,-7.200000e-01
50%,1.471428e+09,3.596970e+03,3.598190e+03,3.595620e+03,3.597000e+03,1.979811e+00,3.810124e+03,3.596804e+03,0.000000e+00
75%,1.544288e+09,8.627270e+03,8.632980e+03,8.621090e+03,8.627160e+03,7.278216e+00,2.569821e+04,8.627637e+03,6.800000e-01
max,1.617149e+09,6.176356e+04,6.178183e+04,6.167355e+04,6.178180e+04,5.853852e+03,1.390067e+07,6.171621e+04,1.871980e+03


### 2.2 Data Quality

In [27]:
print(df.isnull().sum())

Timestamp                  0
Open                 1243608
High                 1243608
Low                  1243608
Close                1243608
Volume_(BTC)         1243608
Volume_(Currency)    1243608
Weighted_Price       1243608
Day_Stats            1243608
DS_categorical       1243608
dtype: int64


From this function we can see that there is significant number of null values (NaN) within our dataset. We have decided that elimination would be a be a better solution than imputation in order to smooth out our data and make our analysis easier. The main reason for this is the sheer number of missing data points it would be nearly impossible to impute these values in an effective manner.

In [18]:
# Remove rows where all of the values are NaN
df_eliminated = df.dropna(axis=0,thresh=2)

# print number of rows in df_imputed
print('Length of updated dataset:', df_eliminated.shape[0])

# Print the number of rows that have been deleted by the elimination
print('Number of rows deleted by elimination operation:', df.shape[0]-df_eliminated.shape[0])

df_eliminated.head()

Length of updated dataset: 3613769
Number of rows deleted by elimination operation: 1243608


,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Day_Stats,DS_categorical
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000,0.00,No Change
478,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000,0.00,No Change
547,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411,-0.07,No Change
548,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000,0.00,No Change
1224,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000,0.00,No Change


In [20]:
print(df_eliminated.isnull().sum())

Timestamp            0
Open                 0
High                 0
Low                  0
Close                0
Volume_(BTC)         0
Volume_(Currency)    0
Weighted_Price       0
Day_Stats            0
DS_categorical       0
dtype: int64


After performing the elimination operation, approximately 25% of the rows were eliminated giving us a much cleaner and more functional data set.  Additionally, the .isnull() function shows us that there are no more data holes within any of our features so we can proceed comfortably with the remainder of our analysis.

## 3. Data Visualization

### 3.1 Feature Distribution Visualization

### 3.2 Interesting Questions

After using a variety of different plots to dissect and explore our data, we came up with these 3 interesting questions:

    1. 
    2. 
    3.

#### 3.2.1

#### 3.2.2

#### 3.2.3

## 4. Exceptional Work

### References:

[1] Kaggle Bitcoin Historical Data Dataset. https://www.kaggle.com/mczielinski/bitcoin-historical-data